Expenses tracker
Jorge Bueno Pérez

Framework: Flask 
Databases: SQLALchemy and SQLIte3 
Front-end: HTML, CSS and Javascript 
Back-end: Python (pandas, numpy and matplotlib)

The code of the application will be divided in the next parts:

A) Import all the necessary packages
    A1) Flask packages
    A2) Python packages

B) Define app

C) Connection with the database:
	C1) SQLAlchemy and SQLite3
    
D) Database:
	D1) Structure
    D2) Export as .db
    D3) Populate table
    
E)User interface and connections:
	E1) Login and logout pages
	E2) Index page
	E3) Income page
	E4) Outcome page
	E5) Analysis page
	E6) Graphs page
    
F) Final connection with the server

A) Import all the necessary packages

    A1)Flask packages:

In [2]:
from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy 
from flask import g
import sqlite3 

    A2) Python packages:

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mp
import io
import base64
import urllib

B) Define app

In [4]:
app = Flask(__name__)

C) Connection with the database:

    C1) SQLAlchemy:

In [5]:
connection = sqlite3.connect('expenses_tracker.db', check_same_thread=False)
DATABASE = './expenses_tracker.db'
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///expenses_tracker.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

app.secret_key = 'developmentkey'


def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(DATABASE)
    return db

D) Database:

    D1) Structure: It will be dividided in six parts, one for each column

In [6]:
#User has 8 columns:
class User(db.Model):
    __tablename__ = 'user'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(30), unique=True)
    first_name = db.Column(db.String(15), nullable=False)
    second_name = db.Column(db.String(25), nullable=True, default='N/A')
    email = db.Column(db.String(50), nullable=False)
    user_city = db.Column(db.String(15), nullable=True, default='N/A')
    prof_status = db.Column(db.String(15), nullable=True, default='N/A')
    password = db.Column(db.String(100), nullable=False)
    outcomes = db.relationship('Outcome', backref='categories', lazy=True)
    incomes = db.relationship('Income', backref='categories1', lazy=True)

    def __init__(self, username, first_name, second_name, email, user_city, prof_status, password):
        self.username = username
        self.first_name = first_name
        self.second_name = second_name
        self.email = email
        self.user_city = user_city
        self.prof_status = prof_status
        self.password = password

        
        '''Outcomes and incomes were created because other tables (outcomes and incomes) have foreign keys related with the 
table user, it should be defined later.'''
'''
In case of creating a new User the argument will pass to the  __init__ method in order to initialize the object. Then
self, that it represent the instance of the class, bind the attributes with the given new arguments. It will be applied 
the same in every table.
'''

'\nIn case of creating a new User the argument will pass to the  __init__ method in order to initialize the object. Then\nself, that it represent the instance of the class, bind the attributes with the given new arguments. It will be applied \nthe same in every table.\n'

In [7]:
#Category has 2 columns:
class Category(db.Model):
    __tablename__ = 'category'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    category_name = db.Column(db.String(15), nullable=False)
    outcomes1 = db.relationship('Outcome', backref='categories2', lazy=True)
    incomes1 = db.relationship('Income', backref='categories3', lazy=True)

    def __init__(self, category_name):
        self.category_name = category_name
'''
Outcomes1 and incomes1 were created because other tables (outcomes and incomes) have foreign keys related with the 
table category, it should be defined later.
'''

'\nOutcomes1 and incomes1 were created because other tables (outcomes and incomes) have foreign keys related with the \ntable category, it should be defined later.\n'

In [8]:
#Sender has
class Sender(db.Model):
    __tablename__ = 'sender'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    sender_name = db.Column(db.String(15), nullable=False)
    sender_city = db.Column(db.String(15), nullable=True, default='N/A')
    sender_country = db.Column(db.String(15), nullable=True, default='N/A')
    senders = db.relationship('Income', backref='senders', lazy=True)

    def __init__(self, sender_name, sender_city, sender_country):
        self.sender_name = sender_name
        self.sender_city = sender_city
        self.sender_country = sender_country
'''
Senders were created because other table (outcomes) has a foreign key related with the 
table vendor, it should be defined later.
'''

'\nSenders were created because other table (outcomes) has a foreign key related with the \ntable vendor, it should be defined later.\n'

In [9]:
#Vendor has 5 columns:
class Vendor(db.Model):
    __tablename__ = 'vendor'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    vendor_name = db.Column(db.String(15), nullable=False)
    vendor_city = db.Column(db.String(15), nullable=True, default='N/A')
    vendor_country = db.Column(db.String(15), nullable=True, default='N/A')
    vendors = db.relationship('Outcome', backref='vendors', lazy=True)

    def __init__(self, vendor_name, vendor_city, vendor_country):
        self.vendor_name = vendor_name
        self.vendor_city = vendor_city
        self.vendor_country = vendor_country
'''
Vendors were created because other table (outcomes) has a foreign key related with the 
table vendor, it should be defined later.
'''

'\nVendors were created because other table (outcomes) has a foreign key related with the \ntable vendor, it should be defined later.\n'

In [10]:
#Outcome has 10 columns:

class Outcome(db.Model):
    __tablename__ = 'outcome'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    ID_user = db.Column(db.Integer, db.ForeignKey('user.id'), 
                        nullable=False) #Outcomes
    ID_vendor = db.Column(db.Integer, db.ForeignKey('vendor.id'),
                          nullable=False) #Vendors
    ID_category = db.Column(db.Integer, db.ForeignKey('category.id'),
                            nullable=False) #Outcomes1
    amount_outcome = db.Column(db.DECIMAL(10, 2), nullable=False)
    currency_outcome = db.Column(db.String(3), nullable=False)
    date_outcome = db.Column(db.String(15), nullable=False)
    date_outcome1 = db.Column(db.Integer, nullable=False)
    p_m_outcome = db.Column(db.String(20), nullable=False)
    comment_outcome = db.Column(db.String(255), nullable=True, default='N/A')

    def __init__(self, ID_user, ID_vendor, ID_category, amount_outcome,
                 currency_outcome, date_outcome, p_m_outcome, comment_outcome, date_outcome1):
        self.ID_user = ID_user
        self.ID_vendor = ID_vendor
        self.ID_category = ID_category
        self.amount_outcome = amount_outcome
        self.currency_outcome = currency_outcome
        self.date_outcome = date_outcome
        self.date_outcome1 = date_outcome1
        self.p_m_outcome = p_m_outcome
        self.comment_outcome = comment_outcome
'''There are three foreign keys:
    1)outcome.ID_user = user.id (Outcomes)
    2)outcome.ID_vendor = vendor.id (Vendors)
    3)outcome.ID_categroy = category.id (Outcomes1)
'''

'There are three foreign keys:\n    1)outcome.ID_user = user.id (Outcomes)\n    2)outcome.ID_vendor = vendor.id (Vendors)\n    3)outcome.ID_categroy = category.id (Outcomes1)\n'

In [11]:
#Income has 10 columns:

class Income(db.Model):
    __tablename__ = 'income'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    ID_user = db.Column(db.Integer, db.ForeignKey('user.id'),
                        nullable=False) #Incomes
    ID_sender = db.Column(db.Integer, db.ForeignKey('sender.id'),
                          nullable=False) #Senders
    ID_category = db.Column(db.Integer, db.ForeignKey('category.id'),
                            nullable=False) #Incomes1
    amount_income = db.Column(db.DECIMAL(10, 2), nullable=False)
    currency_income = db.Column(db.String(3), nullable=False)
    date_income = db.Column(db.String(15), nullable=False)
    date_income1 = db.Column(db.Integer, nullable=False)
    p_m_income = db.Column(db.String(15), nullable=False)
    comment_income = db.Column(db.String(255), nullable=True, default='N/A')

    def __init__(self, ID_user, ID_sender, ID_category, amount_income,
                 currency_income, date_income, p_m_income, comment_income, date_income1):
        self.ID_user = ID_user
        self.ID_sender = ID_sender
        self.ID_category = ID_category
        self.amount_income = amount_income
        self.currency_income = currency_income
        self.date_income = date_income
        self.date_income1 = date_income1
        self.p_m_income = p_m_income
        self.comment_income = comment_income
'''There are three foreign keys:
    1)incomes.ID_user = user.id (Incomes)
    2)incomes.ID_sender = sender.id (Senders)
    3)incomes.ID_categroy = category.id (Incomes1)
'''

'There are three foreign keys:\n    1)incomes.ID_user = user.id (Incomes)\n    2)incomes.ID_sender = sender.id (Senders)\n    3)incomes.ID_categroy = category.id (Incomes1)\n'

    D2) Export as .db:

We should run in the Terminal/CMD, the below code:

python
rom app import db
db.create_all()

    D3) Populate tables:

Once the file expenses_tracker.db has been created, we should populate the tables: User, Category, Vendor and Sender
To do this I creatd the SQL script: create_values.sql, and I ran the script with the help of sqlite3:

import sqlite3
connection = sqlite3.connect('expenses_tracker.db')


def scriptexecution(filename):
    with open(filename, 'r') as s:
        sql_script = s.read()
        connection.executescript(sql_script)
    s.closed


scriptexecution('create_values.sql')

The above code was created as a Markdownd, as the tables were already populated

E)User interface and connections:

    E1) Login and logout pages

In [12]:
@app.teardown_appcontext
def close_connection(exception):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

In [13]:
def query_db(query, args=(), one=False):
    cur = get_db().execute(query, args)
    rv = cur.fetchall()
    cur.close()
    return (rv[0] if rv else None) if one else rv

In [14]:
def valid_login(username, password):
    user = query_db('select * from User where username = ? and password = ?',
                    [username, password], one=True)
    if user is None:
        return False
    else:
        return True

@app.route("/")
@app.route('/login', methods=['POST', 'GET'])
def login():
    error = None
    if request.method == 'POST':
        if valid_login(request.form['username'], request.form['password']):
            return log_the_user_in(request.form['username'])
        else:
            error = 'Invalid username/password'
    return render_template('login.html', error=error)

In [15]:
@app.route("/logout.html")
def logout():
    return render_template('login.html')

The above code check in the login page the username and password introduced is the same as the one in the table User from our database, in case the user or password is not correct the user will receive the mesagge: Invalid username/password. 

Also in the HTML.file: login.html it has been introduced a JavaScript function, in order to evaluate if the user submitted a username and password, because if the user just submitted one of them, the website will display an error mesagge.

The logout it is a simple redirection to the login page.

    E2) Index page

In [16]:
@app.route('/index.html')
def index1():
    return render_template('index.html')

There is nothing special in the index, the user has a short description with the main funtionalities, also a menu in the left side in order to navigate through the diferent pages.

    E3) Income page

In [17]:
@app.route('/data_form_income.html', methods=['GET', 'POST'])
def data_form_income():

    if request.method == 'POST':
        newID_user = request.form['ID_user']
        newID_sender = request.form['ID_sender']
        newID_category = request.form['ID_category']
        newamount_income = request.form['amount_income']
        newcurrency_income = request.form['currency_income']
        newdate_income = request.form['date_income']
        newdate_income1 = request.form['date_income1']
        newp_m_income = request.form['p_m_income']
        newcomment_income = request.form['comment_income']
        newIncome = Income(newID_user, newID_sender, newID_category, newamount_income,
                           newcurrency_income, newdate_income, newp_m_income, newcomment_income, newdate_income1)
        db.session.add(newIncome) #Just run the query in the session
        db.session.commit() #The data from the session is stored into the database
        return render_template('/index.html') #Once the data is introduced, the user will retuen to the index page
    else:
        return render_template('data_form_income.html')

In this website the user will be able to select several data from a drop down list, then it should be run as a query and inroduced in the database.

The user will be able to store in every query: User, Sender, Category, Amount, Currency, Month, Year, Payment method and an optional comment.

It has been used the flask method: POST, this means that the user submit a request to introduce data into the database.

    E4) Outcome page:

In [18]:
@app.route('/data_form_outcome.html', methods=['GET', 'POST'])
def data_form_outcome():

    if request.method == 'POST':
        newID_user = request.form['ID_user']
        newID_vendor = request.form['ID_vendor']
        newID_category = request.form['ID_category']
        newamount_outcome = request.form['amount_outcome']
        newcurrency_outcome = request.form['currency_outcome']
        newdate_outcome = request.form['date_outcome']
        newdate_outcome1 = request.form['date_outcome1']
        newp_m_outcome = request.form['p_m_outcome']
        newcomment_outcome = request.form['comment_outcome']
        newOutcome = Outcome(newID_user, newID_vendor, newID_category, newamount_outcome,
                             newcurrency_outcome, newdate_outcome, newp_m_outcome, newcomment_outcome, newdate_outcome1)
        db.session.add(newOutcome)
        db.session.commit()
        return render_template('/index.html')
    else:
        return render_template('data_form_outcome.html')

In this website the user will be able to select several data from a drop down list, then it should be run as a query and inroduced in the database.

The user will be able to store in every query: User, Vendor, Category, Amount, Currency, Month, Year, Payment method and an optional comment.

It has been used the flask method: POST, this means that the user submit a request to introduce data into the database.

    E5) Analysis page:

In [19]:
@app.route('/analysis.html')
def analysis():
    return render_template('analysis.html')

I would like to mentioned that it has been created only for one user: jorge10:

In [20]:
@app.route('/jorge10.html', methods=['GET', 'POST'])
def analysisjorge10():
    db = get_db()
    cur = db.execute('SELECT(val1 - val2) \
                     FROM(SELECT ID_user, currency_income, sum(amount_income) AS val1 \
                          FROM income \
                          GROUP BY ID_user) inc \
                     JOIN(SELECT ID_user, currency_outcome, sum(amount_outcome) AS val2 \
                          FROM outcome \
                          GROUP BY ID_user) out \
                     ON(inc.ID_user=out.ID_user) \
                     WHERE out.ID_user=10 \
                        AND out.currency_outcome="pln" \
                        AND inc.currency_income="pln"') #SQLAlchemy - Current balance - value

    cur1 = db.execute('SELECT sum(outcome.amount_outcome) \
                        FROM outcome \
                        INNER JOIN user, vendor \
                            ON outcome.ID_vendor=vendor.id \
                            AND outcome.ID_user=user.id \
                        WHERE user.username="jorge10" \
                            AND vendor.vendor_name="Biedronka"') #SQLAlchemy - Total spent in Biedronka - value

    cur2 = db.execute('SELECT SUM(outcome.amount_outcome) \
                      FROM outcome \
                      INNER JOIN category \
                        ON outcome.ID_category=category.id \
                      WHERE outcome.ID_user=10 \
                        AND category.category_name="Parties" \
                        AND outcome.date_outcome="Dec" \
                        AND outcome.date_outcome1="2019" \
                        AND outcome.currency_outcome="pln"') 
                        #SQLAlchemy - Total spent in parties in December 2019 - value

    cur3 = db.execute('SELECT ROUND(AVG(outcome.amount_outcome), 2) \
                        FROM outcome \
                        WHERE outcome.ID_category=25 \
                            AND outcome.currency_outcome="pln"') #SQLAlchemy - Average spent in rent - value

    df3 = pd.read_sql_query('SELECT outcome.amount_outcome, outcome.currency_outcome, \
                    category.category_name, vendor.vendor_name \
                        FROM outcome \
                        INNER JOIN category, vendor, user \
                            ON outcome.ID_category=category.id \
                            AND outcome.ID_vendor=vendor.id \
                            AND outcome.ID_user=user.id \
                        WHERE user.id=10 \
                            AND outcome.date_outcome="Dec" \
                            AND outcome.date_outcome1=2019 '\
                            , connection) #SQLite3 - pandas - Outcomes introduced in December 2019 - table

    df3.rename(columns={'amount_outcome': 'Amount', 'currency_outcome': 'Currency',
                        'category_name': 'Category', 'vendor_name': 'Vendor'}, inplace=True)
                        #In order to change the name 

    balance = cur.fetchall()
    out_cat = cur1.fetchall()
    out_cat1 = cur2.fetchall()
    avg_out = cur3.fetchall()

    return render_template('jorge10.html', balance=balance, out_cat=out_cat,
                           out_cat1=out_cat1, avg_out=avg_out, tables=[
                               df3.to_html(classes='data')],
                           titles=df3.columns.values)

It has been created 5 diferent SLQ queries:
    1)Current balance - value
    2)Total spent in Biedronka - value
    3)Total spent in parties in December 2019 - value
    4)Average spent in rent - value
    5)Outcomes introduced in December 2019 - table

It has been used the flask method: GET, in this case the user run a request to the server, in order to have some data from the database.

    E6) Graphs page:

In [21]:
@app.route("/graphs.html")
def graphs():
    return render_template('graphs.html')

I would like to mentioned that it has been created only for one user: jorge10:

In [22]:
@app.route('/graphs1.html')
def build_plot():
    df = pd.read_sql_query('SELECT  date_outcome, SUM(outcome.amount_outcome) \
                       FROM    outcome \
                       INNER JOIN user \
                       on outcome.ID_user=user.id \
                       WHERE user.username="jorge10" \
                       AND outcome.amount_outcome \
                       AND outcome.date_outcome1=2019\
                       GROUP BY outcome.date_outcome',
                           connection) ##SQLite3 - pandas - Total outcomes per month in 2019 - table
    df.rename(columns={'date_outcome': 'Month',
                       'SUM(outcome.amount_outcome)': 'Outcome'}, inplace=True)
                        #In order to change the name of the columns of our data frame df
        
    df.loc[df['Month'] == "Jan", 'C'] = 1
    df.loc[df['Month'] == "Feb", 'C'] = 2
    df.loc[df['Month'] == "Mar", 'C'] = 3
    df.loc[df['Month'] == "Apr", 'C'] = 4
    df.loc[df['Month'] == "May", 'C'] = 5
    df.loc[df['Month'] == "Jun", 'C'] = 6
    df.loc[df['Month'] == "Jul", 'C'] = 7
    df.loc[df['Month'] == "Aug", 'C'] = 8
    df.loc[df['Month'] == "Sep", 'C'] = 9
    df.loc[df['Month'] == "Oct", 'C'] = 10
    df.loc[df['Month'] == "Nov", 'C'] = 11
    df.loc[df['Month'] == "Dec", 'C'] = 12
    '''Into the df the data is not ordered by month, it is order by the Outcome.id (the same order as the user 
    introduced the date).

    For this, to order the data frame, the above code created a new column (C) with values a new values, based of the 
    string values of the column Month, by the condition: Jan == 1, Feb == 2 ... like that till the last month.

    Now we will be able to order the data based on the column C, with the below code, then we will create a new 
    dataframe (df2), with the columns that we need (Month and Outcome):
    '''
    dfa = df.sort_values(['C'], ascending=True)
    df2 = dfa[['Month', 'Outcome']].copy()
    
    #Now we will proceed to create a bar chart of our df2, with the help of matplotlib.pyplot
    img = io.BytesIO()
    plt.bar(df2['Month'], df2['Outcome'], color='teal')
    plt.title('Graph 1: Vertical bar chart of otcomes per month in 2019:') #To create tittle
    plt.xlabel('Month (2019)') #To show the value of x
    plt.ylabel('Outcome (PLN)') #To show the value of y
    plt.savefig(img, format='png')
    img.seek(0)

    plot_url = base64.b64encode(img.getvalue()).decode()
    
    return '<img src="data:image/png;base64,{}">'.format(plot_url)
    #Finally we will go to a new website, where the user will be only able to see the graph

It has been created one SQL query: Total outcomes per month in 2019

The result was store in the data frame: df, created with pandas.

It has been used the flask method: GET, in this case the user run a request to the server, in order to have some data from the database.

It was not declarated in @app.route, as this method is predeterminate.

F) Final connection with the server

Finally we can run our application into the server with the below code.

Please note that I included the below code as a comment, because some issues with the server happened. I will recomend to run the app from the original python file: app.py, intead of running this jupyer notebook.

In [23]:
#if __name__ == "__main__":
    #app.run(debug=True) 
#With debug as True, it is not needed to restar the server when something was changed